In [3]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

!pip install --quiet huggingface-hub evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00


In [4]:
from huggingface_hub import login
login()

In [5]:
!nvidia-smi

Sun Nov 12 22:29:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
!pip install --quiet pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00


In [8]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [9]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U datasets evaluate #scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.01, shuffle=False)
train_dataset = dataset["train"]
test_dataset = dataset["test"].train_test_split(test_size=0.5, shuffle=False)
eval_dataset = test_dataset["test"]
test_dataset = test_dataset["train"]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [11]:
import json
import random
print(train_dataset)
print(eval_dataset)
print(test_dataset)
print(json.dumps(eval_dataset[random.randrange(len(eval_dataset))], indent=2))
print(eval_dataset[random.randrange(len(eval_dataset))].keys())

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 180993
})
Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 915
})
Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 914
})
{
  "id": "85617db2-cbb6-4de7-9ed6-ebc2bd541755",
  "question": "Following delivery, uterine fundus lies about:",
  "opa": "10 1/2 \" above pubic symphysis",
  "opb": "8 1/2 \" above pubic symphysis",
  "opc": "5 1/2 \" above pubic symphysis",
  "opd": "2 1/2 \" above pubic symphysis",
  "cop": 2,
  "choice_type": "multi",
  "exp": "Ans. C. 5 1/2 above pubic symphysisFollowing delivery, the fundus lies about 13.5cm(51/2) above the pubic symphysis. During the first 24 hrs the level remains constant, thereafter there is steady decrease i

In [12]:
def generate_question(data_point):
    question = f""" {data_point['question']} [0] {data_point['opa']} [1] {data_point['opb']} [2] {data_point['opc']} [3] {data_point['opd']}
    """
    return question

def generate_prompt(data_point):
    full_prompt=f"""You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.\n### Question: {generate_question(data_point)}\n### Answer: """

    if data_point["cop"] != "" and data_point["exp"] != "":
        full_prompt= full_prompt + f"""{data_point["cop"]}\n### Explanation: {data_point["exp"]}"""

    return full_prompt

generate_prompt(eval_dataset[random.randrange(len(eval_dataset))])

'You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.\n### Question:  Parietal peritonium is lined by ? [0] Simple squamous [1] Stratified squamous [2] Cuboidal [3] Columnar\n    \n### Answer: 0\n### Explanation: A i.e. Simple squamousTransitional epithelium is a multilayered (4 - 6) stretchable epithelium, which has extra reserve of cell layers Q i.e. as tension increases the epithelial sheet is expanded & number of observable cell layers decrease & cell shape change from cuboid to squamous.The deepest layer of transitional epithelium is columnar or cuboidal; middle layer is polyhedral or pear shaped & surface layer is umbrella shaped.It differs from stratified squamous epithelium in that the cells at surface are not squamous.Membranous and most of penile urethraQ (in males) is lined by pseudostratified or stratified columnar epitheliumAnsa nephroni (renal loop of Henle) and mesothelium (i.e. serous/outer/parietal layer

In [13]:
print_gpu_utilization()

GPU memory occupied: 233 MB.


In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [15]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    model_max_length=4096,
    device_map="cuda:0",
    add_eos_token=False)

tokenizer.pad_token = tokenizer.eos_token
print_gpu_utilization()

GPU memory occupied: 5747 MB.


In [23]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

def tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    return tokenize(full_prompt)

def generate_tokens(m, prompt, max_new_tokens=32):
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    m.eval()
    with torch.no_grad():
        return tokenizer.decode(m.generate(**model_input, max_new_tokens=max_new_tokens, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01, eos_token_id=tokenizer.eos_token_id)[0].cuda())

In [26]:
sample = test_dataset[1].copy()
print(f"answer {sample['cop']}")
print("----------------")
sample['cop'] = ''
sample['exp'] = ''
print(generate_tokens(base_model, generate_prompt(sample), 64))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer 0
----------------
<s> You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.
### Question:  Primary cause of bleeding disorder in liver damage is due to [0] decreased level of prothrombin [1] Lack of vitamin K [2] Platelet deficiency [3] Lack of vitamin B
    
### Answer:  [1] Lack of vitamin K

### Explanation:

Vitamin K is a fat-soluble vitamin that is essential for the synthesis of several coagulation factors. Vitamin K is required for the synthesis of prothrombin, which is a pre


In [18]:
new_model = "MedMistral-7B"
base_model_id = f"matthewchung74/{new_model}"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

hf_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,
#     quantization_config=bnb_config,
#     device_map="cuda:0",
#     trust_remote_code=True,
#     use_auth_token=True)


# new_model = "MedMistral-7B"
# hf_model = PeftModel.from_pretrained(base_model, f"matthewchung74/{new_model}",
#     device_map="cuda:0",
#     trust_remote_code=True,
#     use_auth_token=True)

In [27]:
sample = test_dataset[1].copy()
print(f"answer {sample['cop']}")
print("----------------")
sample['cop'] = ''
sample['exp'] = ''
print(generate_tokens(hf_model, generate_prompt(sample), 64))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer 0
----------------
<s> You are a helpful medical assistant. Your task is to answer the following question one of the options and explain why.
### Question:  Primary cause of bleeding disorder in liver damage is due to [0] decreased level of prothrombin [1] Lack of vitamin K [2] Platelet deficiency [3] Lack of vitamin B
    
### Answer: 0
### Explanation: None of the options are correct.
The primary cause of bleeding disorder in liver damage is due to decreased level of prothrombin.
Liver is the main site of synthesis of prothrombin.
Prothrombin is synthesized in the end


In [28]:
def parse_answer(response):
    s = response.split("### Answer:")[1]
    s = s.split("### Explanation")[0].strip()[0]
    return int(s)

In [29]:
refs = []
predictions = []
for s in test_dataset:
    ref = s['cop']
    refs.append(ref)
    s['cop'] = ''
    s['exp'] = ''
    response = generate_tokens(hf_model, generate_prompt(s), 64)
    predict = parse_answer(response)
    predictions.append(predict)
    print(f"{ref} {predict}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

3 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 3
0 3


In [30]:
import evaluate
accuracy = evaluate.load("accuracy")
accuracy.add_batch(references=refs, predictions=predictions)
accuracy.compute()

{'accuracy': 0.7210065645514223}

In [31]:
refs = []
predictions = []
for s in eval_dataset:
    ref = s['cop']
    refs.append(ref)
    s['cop'] = ''
    s['exp'] = ''
    response = generate_tokens(hf_model, generate_prompt(s), 64)
    predict = parse_answer(response)
    predictions.append(predict)
    print(f"{ref} {predict}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1 1


KeyboardInterrupt: ignored

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")
accuracy.add_batch(references=refs, predictions=predictions)
accuracy.compute()

In [ ]:
# from google.colab import runtime
# runtime.unassign()